### Instructions 

This homework is longer than the other ones - take time to get through it all! 

Work through each section and provide a well-commented response in one or more Python cells. I should be able to execute the notebook and obtain the same results that you show in the cell (_do not clear the cell outputs before uploading your final notebook to Git!_) 

Part of your grade (_see rubric_) is based on preparing a logical notebook that is easy to follow.

### Part 1: Short answer questions 

Explain in your own words the following concepts: 

* Residual squared error 
* Sum of squared residuals 
* $R^2$ statistic 

_note: I expect you to not simply read these definitions in ISL and modestly change the words. Please take a few minutes to read multiple definitions and come up with your own definition_ 

#### Residual Squared Error

The residual error is the difference between the error of all participants in a sample, from the error of the sample mean, or some other quantity of interest. The residual squared error is simply the square of this difference.

### Sum of squared residuals

The sum of squared residuals is the sum of the squared difference between an observable value and some estimated value of a qunatity of interest.

### R squared statistic

The R squared statistic is a measure of the accuracy of data that is fitted to a regression line

### Part 2: Logistic Regression

Implement both the linear and logistic regression models using sklearn, calculate the coefficients $\beta_0$ and $\beta_1$ and exactly (to the best of your ability) reproduce Figure 4.2 in the book.  

### Part 3: Simple Linear Regression

1. Make sure you understand all the components of the SLR model Chad showed in class and code the same thing yourself 

2. Generate a separate set of training data (25 points) and validation data (15 poins).  Each data set should have the same amount of irreducible error (random noise term) applied to it

3. Make a plot of the error in the testing data as a function of the random strength. There is no "one answer" for this part, open ended graded based on creativity and quality of results 

4. For the exact example I showed in class, calculate the RSE, SSR, $R^2$, 95% confidence intervals for both $\beta_i$, and evaluate the P-value at 0.05 significance level for $\beta_1$

5. Explain in plain language the meaning of the P-value test 

### Part 4: Multiple Linear Regression
1. The code below (next cell) can be used with the Harvard dataset (HCEPD_100K.csv) used in DSMCER HW2 - it is based on the MLSR of the harvard data. Some information: 
    * Your colleague gives you this code makes a 3x1 composite plot  
    * They were trying to look at individual effects ($X_i$) values vs. the PCE to try and understand how important the individual $\beta_i$ are  
    * This approach is clearly wrong, both in concept as well as in code execution. Offer some explanations why. 
    * In CH3 of ISL, the F-test is explained for the multiple linear regression case. Calculate the F-statistics and explain in plain language the conclusion
    * Calculate the p-values for each of the $\beta$ values

In [ ]:

print(regr2.coef_)
print(regr2.intercept_)

# generate matrix X to make predictions of PCE over the X parameter space 
pts=100
X=np.zeros((pts,3))
X[:,0]=np.linspace(harvard.mass.min(),harvard.mass.max(),pts)
X[:,1]=np.linspace(harvard.voc.min(),harvard.voc.max(),pts)
X[:,2]=np.linspace(harvard.e_lumo_alpha.min(),harvard.e_lumo_alpha.max(),pts)

# plot the predicted data 
plt.subplot(311)
plt.scatter(harvard.mass,harvard.pce)
plt.plot(X[:,0],regr2.predict(X),color='red',lw='3')
plt.ylabel('PCE')
plt.xlabel('$mass$')

plt.subplot(312)
plt.scatter(harvard.voc,harvard.pce)
plt.plot(X[:,1],regr2.predict(X),color='red',lw='3')
plt.ylabel('PCE')
plt.xlabel('$VOC$')

plt.subplot(313)
plt.scatter(harvard.e_lumo_alpha,harvard.pce)
plt.plot(X[:,2],regr2.predict(X),color='red',lw='3')
plt.ylabel('PCE')
plt.xlabel('$E_{LUMO}$')



### Part 5: Resampling methods 

I would like you to write a clearly labeled function that performs bootstrapping using a subset of the HCEPD_100K.csv data. Your function should be versatile in terms of how much of the data is selected for the bootstrap and and how many iterations are run.  

1. Prepare a plot of MSE (using the same 3-parameter PCE fit (using mass/VOC/lumo features). The x-axis should be the number of bootstrap samples and the y-axis should be the MSE. You should show a line for boostrap samples sizes of 100, 1000, and 5000.  
    
2. For the same sampling you should show a plot the estimate of the three relevant beta coefficients 
    